<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_Style_Transfer_AudioLDM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Check GPU

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
#@title Install AudioLDM2 Dependencies
import subprocess, os, sys, ipykernel

!pip install -q condacolab
import condacolab
condacolab.install()

res = subprocess.run(['conda', 'create', '-n' 'audioldm', 'python=3.8'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(res)

res = subprocess.run(['conda', 'activate', 'audioldm'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(res)
#!pip uninstall audioldm --yes


res = subprocess.run(['pip', 'install', 'git+https://github.com/haoheliu/AudioLDM2.git'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(res)

#gitclone('https://github.com/haoheliu/AudioLDM')

print('install complete')








In [ ]:
#@title Generate Audio


!apt-get install ffmpeg

try:
    import dawnet_client
except ImportError:
  !pip install dawnet-client --upgrade


import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath

import subprocess, os, sys, ipykernel

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "Enter the token generated by the DAWNet plugin" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()


model_path = os.path.join(
            os.path.expanduser("~"),
            ".cache/audioldm/audioldm-l-full.ckpt",
        )

print('MODEL EXISTS:', str(os.path.exists(model_path)))

outputs_per_input = 3
batch_size = 1
candidates = 4
transfer_styles = ['Energetic','Romantic','Melancholic','Nostalgic','Dreamy','Intense','Mysterious','Dark','Playful','Happy','Joyful','Sad','Hopeful','Excited','Sentimental','Calm','Peaceful','Serene','Relaxing','Soothing','Inspirational','Empowering','Motivational','Triumphant','Ethereal','Haunting','Enchanting','Passionate','Soulful','Sensual','Sexy','Funky','Groovy','Jazzy','Bluesy','Rocking','Driving','Explosive','Dynamic','Chill','Laid-back','Whimsical','Quirky','Eccentric','Wistful','Thoughtful','Contemplative','Reflective','Optimistic','Pensive','Fierce','Bold','Majestic','Grand','Regal','Epic','Cinematic','Trippy','Psychedelic','Futuristic','Industrial','Eerie','Creepy','Suspenseful','Tense','Edgy','Menacing','Aggressive','Angry','Frustrated','Resentful','Bittersweet','Disillusioned','Discontent','Disappointed','Conflicted','Desperate','Dramatic','Emotional','Fateful','Heartfelt','Heartbroken','Lonely','Nervous','Paranoid','Rebellious','Sentimental','Skeptical','Stressed','Suspicious','Tender','Tragic','Unsettling','Wary','Yearning','Zealous','Ambitious','Arrogant','Confident','Proud','Upbeat pop','Ethereal electronic music','Heavy metal guitar','Soothing melodies','Driving techno beats','Tender acoustic guitar','Catchy indie rock','Soulful jazz','Beautiful orchestral score','Funky bassline','Haunting ballad','Powerful hip-hop lyrics','Catchy hooks','Sultry vocals','Dreamlike atmosphere','Bluesy guitar solo','Harmonized a cappella','Experimental soundscapes','Atmospheric post-rock','Lyrical folk music','Rhythmic dance beats','Emotional piano ballads','Trippy psychedelic rock','Cheesy 80s pop','Jazzy saxophone solos','Groovy basslines','Fast-paced punk rock','Funky horn section','Soulful backing vocals','Nostalgic 90s hits','Experimental electronic beats','Epic movie soundtracks','Haunting choir vocals','Acoustic singer-songwriter','Hypnotic trance music','Melancholic indie pop','Aggressive rap verses','Atmospheric ambient music','Classic rock guitar riffs','Romantic love songs','Lively Caribbean rhythms','Percussive African beats','Energetic ska music','Patriotic military marches','Bittersweet acoustic ballads','Laid-back reggae vibes','Atmospheric post-punk','Moody synthpop','Spooky Halloween music','Uplifting motivational songs','Dreamy shoegaze guitar','Funky disco beats','Melodic alternative rock','Cheeky novelty songs','Hard-hitting drum and bass','Soulful Motown-inspired tunes','Ethereal choral music','Moody trip-hop','Latin-inspired salsa music','Calming new age music','Bright and sunny pop','Catchy synthwave tunes','Emotional country ballads','Angry punk anthems','Dreamlike indie folk','Slick hip-hop beats','Nostalgic retro sounds','Atmospheric black metal','Bouncy bubblegum pop','Heavy industrial beats','Uplifting gospel music','Progressive rock epics','Catchy Britpop anthems','Nostalgic 2000s hits','Soulful blues guitar','Soaring power ballads','Hypnotic sitar music','Intense heavy rock','Trippy ambient techno','Dreamy indie pop','Jazzy piano solos','Lively mariachi music','Catchy synthpop','Epic cinematic orchestration','Haunting piano melodies','Experimental noise music','Joyful Christmas carols','Upbeat disco funk','Melodic power metal','Cheesy boy band pop','Reflective folk rock','Groovy funk rock','Emotional post-hardcore','Bouncy ska punk','Patriotic American folk songs','Smooth R&B vocals','Atmospheric trip-hop beats','Upbeat electro swing','Catchy garage rock','Energetic dancehall reggae','Heavy doom metal','Heartfelt acoustic covers','Nostalgic synthpop','Piano','Guitar','Drums','Bass guitar','Violin','Cello','Trumpet','Saxophone','Clarinet','Flute','Harmonica','Accordion','Banjo','Mandolin','Ukulele','Harp','French horn','Trombone','Tuba','Oboe','Bassoon','Bagpipes','Didgeridoo','Bongos','Congas','Timpani','Maracas','Tambourine','Cowbell','Guiro','Djembe','Steel drums','Xylophone','Vibraphone','Marimba','Glockenspiel','Triangle','Chimes','Celesta','Organ','Synthesizer','Sampler','Theremin','Electronic drums','Electric guitar','Bass synthesizer','Double bass','Electric violin','Viola','Piccolo','Recorder','Bass clarinet','Euphonium','Sitar','Dulcimer','Zither','Autoharp','Hurdy-gurdy','Harpsichord','Clavichord','Mellotron','Fiddle','Mandola','Lute','Shamisen','Koto','Balalaika','Saz','Darbuka','Cymbals','Gong','Taiko drum','Rainstick','Ocean drum','Kalimba','Finger cymbals','Anklung','Jaw harp','Kalimba','Kazoo','Nose flute','Glass harmonica','Slide whistle','Swanee whistle','Bird whistle','Whamola','Theremin cello','Waterphone','Hulusi','Pungi','Crumhorn','Serpent','Sousaphone','Piccolo trumpet','Ocarina','Pan flute','Ney','Udu','Steel guitar','Chapman Stick','Acoustic','Alternative','Ambient','Americana','Avant-garde','Baroque','Big Band','Bluegrass','Blues','Bollywood','Bossa Nova','Celtic','Chamber','Chant','Choral','Classical','Country','Dance','Disco','Dixieland','Dream Pop','Drone','Dubstep','Electronic','Experimental','Flamenco','Folk','Funk','Gospel','Gothic','Grunge','Hard Rock','Heavy Metal','Hip Hop','House','Indie','Industrial','Jazz','Jungle','Klezmer','Latin','Lounge','Medieval','Metal','Minimalist','Motown','Musique Concrète','New Age','New Wave','Noise','Opera','Orchestral','Pop','Post Punk','Progressive','Psychedelic','Punk','R&B','Ragtime','Rap','Reggae','Renaissance','Rock','Rockabilly','Salsa','Samba','Shoegaze','Singer-Songwriter','Ska','Smooth Jazz','Soul','Southern Rock','Space Rock','Surf','Swing','Synth-pop','Techno','Trance','Trip Hop','Tropical','UK Garage','World','Worldbeat','Zouk','Afrobeat','Alternative Country','Balearic Beat','Barbershop','Bollywood Pop','Cumbia','Dancehall','Deep House','Electro Swing','Emo','Fado','Folktronica','Garage Rock','Glitch','Grindcore','Hardcore Punk']
transfer_strength = '0.5'
output_dir = '/transfer_output'

async def dawnet_func(input_file: DAWNetFilePath):

  try:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    seed = str(random.randint(1, 1000))

    print('INPUT_PATH: ' + str(input_file))
    print('OUTPUT_DIR: ' + str(output_dir))
    print('AUDIO_LEN: ' + str(audio_len))
    print('SEED: ' + seed)
    print('TRANSFER_STYLE: ' + transfer_style)
    print('BATCH SIZE: ' + str(batch_size))
    print('CANDIDATES: ' + str(candidates))

    print('RUN-START')
    # env = os.environ.copy()
    # res = subprocess.run(['audioldm','--mode','transfer','--file_path',"'" + input_path + "'",'--save_path', "'" + output_dir + "'",'-t',transfer_style,'--transfer_strength', str(transfer_strength),'--seed', str(seed)], shell=True, env=env).stdout.decode('utf-8')

    cmd = "audioldm --mode transfer --file_path '{}' --save_path '{}' -t '{}' --transfer_strength {} --duration {} --seed {} --n_candidate_gen_per_text {} --batchsize {} --model_name audioldm-l-full".format(input_path, output_dir, transfer_style, transfer_strength, audio_len, seed, candidates, batch_size)

    !{cmd}

    print(res)

    # after executing your custom code you send data back to the plugin like so ..
    #await dawnet.output().add_file(input_file)
    await dawnet.output().add_message("Returning the input file")
    await dawnet.output().send()

    return True

  except Exception as e:
    print(f"Error in arbitrary_method: {e}")
    return False


dawnet.set_token(token=dawnet_token)
dawnet.set_name("AudioLDM2 - Style Transfer")
dawnet.set_description("This is a template for creating a DAWNet Remote")
dawnet.register_method(dawnet_func)

dawnet.connect_to_server()








# from IPython.display import Audio
# from IPython.display import display
# import csv
# import wave
# import contextlib
# import random


# mount_drive = '/content/drive/MyDrive/SNS_DIFFUSION'



# print('MODEL EXISTS', str(os.path.exists(model_path)))


# outputs_per_input = 10
# batch_size = 1
# candidates = 6
# transfer_styles = ['Energetic','Romantic','Melancholic','Nostalgic','Dreamy','Intense','Mysterious','Dark','Playful','Happy','Joyful','Sad','Hopeful','Excited','Sentimental','Calm','Peaceful','Serene','Relaxing','Soothing','Inspirational','Empowering','Motivational','Triumphant','Ethereal','Haunting','Enchanting','Passionate','Soulful','Sensual','Sexy','Funky','Groovy','Jazzy','Bluesy','Rocking','Driving','Explosive','Dynamic','Chill','Laid-back','Whimsical','Quirky','Eccentric','Wistful','Thoughtful','Contemplative','Reflective','Optimistic','Pensive','Fierce','Bold','Majestic','Grand','Regal','Epic','Cinematic','Trippy','Psychedelic','Futuristic','Industrial','Eerie','Creepy','Suspenseful','Tense','Edgy','Menacing','Aggressive','Angry','Frustrated','Resentful','Bittersweet','Disillusioned','Discontent','Disappointed','Conflicted','Desperate','Dramatic','Emotional','Fateful','Heartfelt','Heartbroken','Lonely','Nervous','Paranoid','Rebellious','Sentimental','Skeptical','Stressed','Suspicious','Tender','Tragic','Unsettling','Wary','Yearning','Zealous','Ambitious','Arrogant','Confident','Proud','Upbeat pop','Ethereal electronic music','Heavy metal guitar','Soothing melodies','Driving techno beats','Tender acoustic guitar','Catchy indie rock','Soulful jazz','Beautiful orchestral score','Funky bassline','Haunting ballad','Powerful hip-hop lyrics','Catchy hooks','Sultry vocals','Dreamlike atmosphere','Bluesy guitar solo','Harmonized a cappella','Experimental soundscapes','Atmospheric post-rock','Lyrical folk music','Rhythmic dance beats','Emotional piano ballads','Trippy psychedelic rock','Cheesy 80s pop','Jazzy saxophone solos','Groovy basslines','Fast-paced punk rock','Funky horn section','Soulful backing vocals','Nostalgic 90s hits','Experimental electronic beats','Epic movie soundtracks','Haunting choir vocals','Acoustic singer-songwriter','Hypnotic trance music','Melancholic indie pop','Aggressive rap verses','Atmospheric ambient music','Classic rock guitar riffs','Romantic love songs','Lively Caribbean rhythms','Percussive African beats','Energetic ska music','Patriotic military marches','Bittersweet acoustic ballads','Laid-back reggae vibes','Atmospheric post-punk','Moody synthpop','Spooky Halloween music','Uplifting motivational songs','Dreamy shoegaze guitar','Funky disco beats','Melodic alternative rock','Cheeky novelty songs','Hard-hitting drum and bass','Soulful Motown-inspired tunes','Ethereal choral music','Moody trip-hop','Latin-inspired salsa music','Calming new age music','Bright and sunny pop','Catchy synthwave tunes','Emotional country ballads','Angry punk anthems','Dreamlike indie folk','Slick hip-hop beats','Nostalgic retro sounds','Atmospheric black metal','Bouncy bubblegum pop','Heavy industrial beats','Uplifting gospel music','Progressive rock epics','Catchy Britpop anthems','Nostalgic 2000s hits','Soulful blues guitar','Soaring power ballads','Hypnotic sitar music','Intense heavy rock','Trippy ambient techno','Dreamy indie pop','Jazzy piano solos','Lively mariachi music','Catchy synthpop','Epic cinematic orchestration','Haunting piano melodies','Experimental noise music','Joyful Christmas carols','Upbeat disco funk','Melodic power metal','Cheesy boy band pop','Reflective folk rock','Groovy funk rock','Emotional post-hardcore','Bouncy ska punk','Patriotic American folk songs','Smooth R&B vocals','Atmospheric trip-hop beats','Upbeat electro swing','Catchy garage rock','Energetic dancehall reggae','Heavy doom metal','Heartfelt acoustic covers','Nostalgic synthpop','Piano','Guitar','Drums','Bass guitar','Violin','Cello','Trumpet','Saxophone','Clarinet','Flute','Harmonica','Accordion','Banjo','Mandolin','Ukulele','Harp','French horn','Trombone','Tuba','Oboe','Bassoon','Bagpipes','Didgeridoo','Bongos','Congas','Timpani','Maracas','Tambourine','Cowbell','Guiro','Djembe','Steel drums','Xylophone','Vibraphone','Marimba','Glockenspiel','Triangle','Chimes','Celesta','Organ','Synthesizer','Sampler','Theremin','Electronic drums','Electric guitar','Bass synthesizer','Double bass','Electric violin','Viola','Piccolo','Recorder','Bass clarinet','Euphonium','Sitar','Dulcimer','Zither','Autoharp','Hurdy-gurdy','Harpsichord','Clavichord','Mellotron','Fiddle','Mandola','Lute','Shamisen','Koto','Balalaika','Saz','Darbuka','Cymbals','Gong','Taiko drum','Rainstick','Ocean drum','Kalimba','Finger cymbals','Anklung','Jaw harp','Kalimba','Kazoo','Nose flute','Glass harmonica','Slide whistle','Swanee whistle','Bird whistle','Whamola','Theremin cello','Waterphone','Hulusi','Pungi','Crumhorn','Serpent','Sousaphone','Piccolo trumpet','Ocarina','Pan flute','Ney','Udu','Steel guitar','Chapman Stick','Acoustic','Alternative','Ambient','Americana','Avant-garde','Baroque','Big Band','Bluegrass','Blues','Bollywood','Bossa Nova','Celtic','Chamber','Chant','Choral','Classical','Country','Dance','Disco','Dixieland','Dream Pop','Drone','Dubstep','Electronic','Experimental','Flamenco','Folk','Funk','Gospel','Gothic','Grunge','Hard Rock','Heavy Metal','Hip Hop','House','Indie','Industrial','Jazz','Jungle','Klezmer','Latin','Lounge','Medieval','Metal','Minimalist','Motown','Musique Concrète','New Age','New Wave','Noise','Opera','Orchestral','Pop','Post Punk','Progressive','Psychedelic','Punk','R&B','Ragtime','Rap','Reggae','Renaissance','Rock','Rockabilly','Salsa','Samba','Shoegaze','Singer-Songwriter','Ska','Smooth Jazz','Soul','Southern Rock','Space Rock','Surf','Swing','Synth-pop','Techno','Trance','Trip Hop','Tropical','UK Garage','World','Worldbeat','Zouk','Afrobeat','Alternative Country','Balearic Beat','Barbershop','Bollywood Pop','Cumbia','Dancehall','Deep House','Electro Swing','Emo','Fado','Folktronica','Garage Rock','Glitch','Grindcore','Hardcore Punk']
# transfer_strength = '0.5'
# audio_files_dir = mount_drive + '/training/source_audio/sas_data/'
# output_dir = mount_drive + '/transfer_output'
# csv_path = mount_drive + '/transfer_runstate.csv'


# def print_files_in_directory(directory):
#     for root, dirs, files in os.walk(directory):
#         for file in files:
#             print(os.path.join(root, file))


# def audio_style_transfer(input_path: str, output_dir: str, audio_len:float) -> bool:

#     for _ in range(outputs_per_input):
#       seed = str(random.randint(1, 1000))
#       transfer_style = random.choice(transfer_styles)

#       print('INPUT_PATH: ' + str(input_path))
#       print('OUTPUT_DIR: ' + str(output_dir))
#       print('AUDIO_LEN: ' + str(audio_len))
#       print('SEED: ' + seed)
#       print('TRANSFER_STYLE: ' + transfer_style)
#       print('BATCH SIZE: ' + str(batch_size))
#       print('CANDIDATES: ' + str(candidates))

#       print('RUN-START')
#       try:
#         # env = os.environ.copy()
#         # res = subprocess.run(['audioldm','--mode','transfer','--file_path',"'" + input_path + "'",'--save_path', "'" + output_dir + "'",'-t',transfer_style,'--transfer_strength', str(transfer_strength),'--seed', str(seed)], shell=True, env=env).stdout.decode('utf-8')

#         cmd = "audioldm --mode transfer --file_path '{}' --save_path '{}' -t '{}' --transfer_strength {} --duration {} --seed {} --n_candidate_gen_per_text {} --batchsize {} --model_name audioldm-l-full".format(input_path, output_dir, transfer_style, transfer_strength, audio_len, seed, candidates, batch_size)

#         !{cmd}

#         print(res)
#       except Exception as e:
#         print('ERROR!!: ' + str(e))

#       print('RUN-END')

#       print("OUTPUT:")
#       print_files_in_directory(output_dir)

#       print("MODEL:")
#       print_files_in_directory(output_dir)

#       # res = subprocess.run(['audioldm', '--mode', 'transfer', '--file_path', input_path, '--save_path', output_dir,'-t', transfer_style,'--transfer_strength', transfer_strength,'--duration', str(audio_len),'--seed', seed], stdout=subprocess.PIPE).stdout.decode('utf-8')
#       # print(res)

#     return True

# def read_csv() -> list:
#     filenames = []
#     with open(csv_path, 'r') as f:
#         reader = csv.reader(f)
#         for y in reader:
#             if y[0] != '':
#                 filenames.append(y[0])

#     return filenames


# def write_csv(row_data):
#     with open(csv_path, mode='a') as my_csv:
#         my_csv_writer = csv.writer(my_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#         my_csv_writer.writerow([row_data])

# def has_file_been_processed(filename: str) -> bool:
#     if os.path.exists(csv_path) == False:
#         open(csv_path, 'w').close()

#     filenames = read_csv()
#     return filename in filenames




# for dir_name in os.listdir(audio_files_dir):
#     if "-drum" in dir_name:
#       print('SKIPPING DRUMS FOR NOW')
#       continue

#     dir_path = os.path.join(audio_files_dir, dir_name)
#     if os.path.isdir(dir_path) == False:
#         continue

#     for file_name in os.listdir(dir_path):
#         file_path = os.path.join(dir_path, file_name)
#         if os.path.isdir(file_path) == True:
#             continue

#         extension = os.path.splitext(file_name)[1]
#         if extension != '.wav':
#             continue

#         if has_file_been_processed(file_name):
#             print('FILE ALREADY PROCESSED: ' + str(file_name))
#         else:
#             print('PROCESSING: ' + str(file_path))
#             audio_len = 12.5
#             # with contextlib.closing(wave.open(file_path, 'r')) as f:
#             #     frames = f.getnframes()
#             #     rate = f.getframerate()
#             #     audio_len = frames / float(rate)
#             #     print(audio_len)

#             output_path = os.path.join(output_dir, dir_name)

#             if audio_style_transfer(file_path, output_path, float(audio_len)):
#                 write_csv(file_name)
#             else:
#                 print('ERROR: ' + str(file_name))




